In [1]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import glob
import cv2
import numpy as np
import mediapipe as mp
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
from threading import Thread
from collections import deque
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.utils import to_categorical
import time
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tkinter import messagebox

In [3]:
SAMPLES_PER_GESTURE = 100  # Кількість зразків для кожного жесту
SEQUENCE_LENGTH = 30  # Кількість кадрів у послідовності
DELAY_BETWEEN_SAMPLES = 1  # Затримка між зразками (в секундах)
CONFIDENCE_THRESHOLD = 0.90  # Поріг впевненості для розпізнавання жесту
FONT_PATH = "font/ISOCPEUR.ttf"

GESTURES = [
    # Перші 20 Базових жестів
    "Привіт", 
    "До побачення", 
    "Дякую", 
    "Будь ласка", 
    "Вибачте",
    "Так", 
    "Ні", 
    "Допомога", 
    "Туалет", 
    "Вода", 
    "Їсти",
    "Пити", 
    "Стоп", 
    "Більше", 
    "Все", 
    "Мене звати…",
    "Ім’я", 
    "Друг", 
    "Сім'я", 
    "Я люблю тебе",
    
    # Для презентації
    "Дипломна робота",
    "Презентація PowerPoint", 
    "Сьогодні",
    "Я",
    "Хочу",
    
    #Ввічливі слова
    "Добрий ранок",
    "Добрий день",
    "Добрий вечір",
    "Доброї ночі",
    
    # Часті запитання
    "Як справи?",
    "Що?",
    "Де?",
    "Коли?",
    "Чому?",
    "Хто?",
    "Як?",
    "Скільки?",
    
    # Повсякденні слова
    "Добре",
    "Погано",
    "Можливо",
    "Не знаю",
    "Допоможіть",
    "Чекаю",
    "Побачимося",
    "Зрозумів",
    "Не зрозумів",
    "Повторіть",
    
    # Напрямки
    "Вліво",
    "Вправо",
    "Прямо",
    "Назад",
    
    # Час доби
    "Ранок",
    "День",
    "Вечір",
    "Ніч",
    
    # Люди і стосунки
    "Мама",
    "Тато",
    "Брат",
    "Сестра",
    "Друг",
    "Колега",
    
    # Емоції
    "Радий",
    "Сумно",
    "Страх",
    "Гнів",
    "Спокій",
    "Втома",
    
    # Дії
    "Йти",
    "Бігти",
    "Стояти",
    "Сидіти",
    "Чекати",
    "Говорити",
    "Чути",
    "Дивитись",
    "Писати",
    "Читати",
    "Грати",
    "Працювати",
    "Вчитися",
    "Спати",
    
    # Здоров’я
    "Хворий",
    "Лікар",
    "Ліки",
    "Біль",
    
    # Речі та місця
    "Дім",
    "Робота",
    "Магазин",
    "Школа",
    "Лікарня",
    "Квартира",
    
    # Природа та погода
    "Сонце",
    "Дощ",
    "Сніг",
    "Вітер",
    "Тепло",
    "Холодно",
    
    # Допомога і безпека
    "Пожежа",
    "Поліція",
    "Небезпека",
]

In [4]:
class GestureRecognition:
    def __init__(self, mode="collect", gestures=None,
                 samples_per_gesture=100,
                 save_dir='dataset',
                 font_path="font/ISOCPEUR.ttf",
                 window_scale=1.5,
                 delay_between_samples=0.05,
                 sequence_length=30):
        self.mode = mode
        self.gestures = gestures or []
        self.samples_per_gesture = samples_per_gesture
        self.save_dir = save_dir
        self.font_path = font_path
        self.window_scale = window_scale
        self.delay_between_samples = delay_between_samples
        self.sequence_length = sequence_length

        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7)
        self.mp_draw = mp.solutions.drawing_utils

        if not os.path.exists(self.save_dir):
            os.makedirs(self.save_dir)
        if not os.path.exists(self.font_path):
            raise Exception(f"Шрифт не знайдено: {self.font_path}")

    def process_frame(self, frame):
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)
        all_landmarks = []
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_draw.draw_landmarks(frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)
                all_landmarks.append(hand_landmarks)
        return frame, all_landmarks

    def extract_landmarks(self, hand_landmarks):
        return np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten() if hand_landmarks else None

    def show_on_screen(self, frame, text=None, subtext=None, font_size=36):
        frame_pil = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_im = Image.fromarray(frame_pil)
        draw = ImageDraw.Draw(pil_im)
        rect_height = 70 if (text or subtext) else 45
        draw.rectangle([(0, 0), (frame.shape[1], rect_height)], fill=(245, 218, 11, 255))
        if text:
            font = ImageFont.truetype(self.font_path, font_size)
            draw.text((10, 10), text, font=font, fill=(0, 0, 0))
        if subtext:
            font = ImageFont.truetype(self.font_path, int(font_size * 0.7))
            draw.text((10, 40), subtext, font=font, fill=(0, 0, 0))
        return cv2.cvtColor(np.array(pil_im), cv2.COLOR_RGB2BGR)

    def resize_window(self, frame):
        if self.window_scale != 1:
            new_size = (int(frame.shape[1] * self.window_scale), int(frame.shape[0] * self.window_scale))
            frame = cv2.resize(frame, new_size)
        return frame

In [5]:
class AppGUI:
    def __init__(self, gesture_recognition_class, gestures,
                 samples_per_gesture, delay_between_samples,
                 confidence_threshold, sequence_length,
                 window_scale=1.5):
        self.gr_class = gesture_recognition_class
        self.gestures = gestures
        self.samples_per_gesture = samples_per_gesture
        self.delay_between_samples = delay_between_samples
        self.confidence_threshold = confidence_threshold
        self.sequence_length = sequence_length
        self.window_scale = window_scale

        self.model = None
        self.labels = None

        self.collect_running = False
        self.collect_ready = False
        self.collect_gesture_idx = 0

        self.recognize_running = False
        self.recognize_thread = None

        self.sentence_running = False
        self.sentence_thread = None
        self.builded_sentence = []
        self.sentence_last_word = None

        self.root = tk.Tk()
        self.root.title("Розпізнавання жестової мови")
    
        self.frames = {}
        for F in (MainMenu, CollectFrame, RecognizeFrame, GestureListFrame, SentenceFrame):
            frame = F(parent=self.root, app=self)
            self.frames[F] = frame
            frame.grid(row=0, column=0, sticky="nsew")
        self.show_frame(MainMenu)

    def show_frame(self, frameclass):
        for frame in self.frames.values():
            frame.grid_remove()
        self.frames[frameclass].grid()
        self.frames[frameclass].tkraise()
        self.root.update()

    def run(self):
        self.root.mainloop()

    def train_model(self):
        gr = self.gr_class(
            mode="recognize",
            gestures=self.gestures,
            font_path=FONT_PATH,
            window_scale=self.window_scale
        )
        
        early_stop = EarlyStopping(
            monitor='val_loss',      # Следить за валидационной потерей
            patience=5,              # Ждать 5 эпох без улучшения
            restore_best_weights=True  # Вернуть веса с лучшей валидацией
        )

        X, y = [], []
        for idx, gesture in enumerate(self.gestures):
            gesture_dir = os.path.join('dataset', gesture)
            files = glob.glob(os.path.join(gesture_dir, '*.npy'))
            for f in files:
                data = np.load(f)
                if data.shape[0] == self.sequence_length:
                    X.append(data)
                    y.append(idx)

        if not X:
            messagebox.showerror("Помилка", "Датасет пустий! Зберіть дані перед тренуванням.")
            return

        X = np.array(X)
        y = np.array(y)
        y_onehot = to_categorical(y, num_classes=len(self.gestures))
        input_shape = (self.sequence_length, X.shape[2])

        if os.path.exists("gesture_rnn_model.h5"):
            model = load_model("gesture_rnn_model.h5")
            print("Загружена існуюча модель, продовжуємо навчання...")
            opt = Adam(learning_rate=1e-4)   # Меньшая скорость обучения
            model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        else:
            model = Sequential([
                LSTM(64, return_sequences=True, input_shape=input_shape),
                Dropout(0.2),
                LSTM(64),
                Dropout(0.2),
                Dense(64, activation='relu'),
                Dense(len(self.gestures), activation='softmax')
            ])
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            print("Створена нова модель, починаємо навчання...")
    
        # Обучаем модель (новую или дообучаем существующую)
        history = model.fit(
            X,
            y_onehot,
            epochs=30,
            batch_size=8,
            verbose=0,              # убираем стандартный лог
            validation_split=0,   # % валидации
            callbacks=[early_stop, PrintTwoDecimalsCallback()]  # ранняя остановка и красивый вывод
        )
        print("Навчання завершено.")

        model.save("gesture_rnn_model.h5")
        messagebox.showinfo("Інфо", "Тренування завершено. Модель збережена.")
        self.model = model
        self.labels = self.gestures
    
        self.plot_training_history(history)
    
    def retrain_model_from_scratch(self):
        result = messagebox.askokcancel(
            "Підтвердження",
            "Ви впевнені, що хочете повторно натренувати модель? Існуюча модель буде видалена."
        )
        if result:
            filename = "gesture_rnn_model.h5"
            if os.path.exists(filename):
                os.remove(filename)
            print("Модель видалена. Навчаємо з початку...")
            self.train_model()
        else:
            print("Повторне тренування скасовано користувачем.")

    # ------ Data Collection ------
    def start_collect(self, gesture_idx=None):
        if gesture_idx is not None:
            self.collect_gesture_idx = gesture_idx
        if self.collect_running:
            return
        self.collect_running = True
        self.collect_ready = False
        self.collect_thread = Thread(target=self.collect_data_thread, daemon=True)
        self.collect_thread.start()

    def collect_data_thread(self):
        gr = self.gr_class(
            mode="collect", gestures=self.gestures,
            samples_per_gesture=self.samples_per_gesture,
            font_path=FONT_PATH,
            window_scale=self.window_scale
        )

        gesture_name = self.gestures[self.collect_gesture_idx]
        gesture_dir = os.path.join('dataset', gesture_name)
        if not os.path.exists(gesture_dir):
            os.makedirs(gesture_dir)
        collected = len(glob.glob(os.path.join(gesture_dir, '*.npy')))
        to_collect = self.samples_per_gesture - collected
        if to_collect <= 0:
            self.collect_running = False
            self.frames[CollectFrame].status_label.config(
                text=f"Збір завершено для '{gesture_name}', вже є {collected} із {self.samples_per_gesture}"
            )
            self.frames[CollectFrame].btn_start.config(state=tk.NORMAL)
            self.frames[CollectFrame].btn_stop.config(state=tk.DISABLED)
            return
        
        saved_count = 0
        seq = []
        cap = cv2.VideoCapture(0)

        expected_len = 126  # довжина вектора при 2 руках

        while self.collect_running and saved_count < to_collect:
            ret, frame = cap.read()
            if not ret:
               continue
            frame = cv2.flip(frame, 1)
            frame, hands_all = gr.process_frame(frame)
            vec = []
            if hands_all:
                for hand_landmarks in hands_all:
                    vec.extend(gr.extract_landmarks(hand_landmarks))
                if len(hands_all) == 1:
                    vec.extend([0.] * 63)
            else:
                h, w = frame.shape[:2]
                frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                draw = ImageDraw.Draw(frame_pil)
                draw.rectangle([(0, 0), (w, 70)], fill=(245, 218, 11, 255))
                draw.text((10, 10), "Немає руки", font=ImageFont.truetype(FONT_PATH, 36), fill=(0, 0, 0))
                draw.text((10, 40), f"Покажіть жест: {gesture_name}", font=ImageFont.truetype(FONT_PATH, 28), fill=(0, 0, 0))
                frame_disp = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)
                frame_disp = gr.resize_window(frame_disp)
                cv2.imshow('Збір даних', frame_disp)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                continue

            # Ось ця перевірка додана:
            if len(vec) != expected_len:
                print(f"Пропуск кадру: довжина вектора некоректна {len(vec)} (очікувалась {expected_len})")
                continue  # НЕ додаємо кадр в seq, пропускаємо

            seq.append(vec)

            if len(seq) == self.sequence_length:
                np.save(os.path.join(gesture_dir, f"{collected+saved_count:03d}.npy"), np.array(seq))
                saved_count += 1
                seq = []
                self.frames[CollectFrame].status_label.config(
                    text=f"Збережено: {collected+saved_count}/{self.samples_per_gesture}\n'{gesture_name}'"
                )
                self.root.update()
                time.sleep(self.delay_between_samples)

            frame_disp = gr.show_on_screen(
                frame,
                text=f"Жест: '{gesture_name}'",
                subtext=f"Збережено: {collected+saved_count}/{self.samples_per_gesture} | Кадр: {len(seq)}/{self.sequence_length}"
            )
            frame_disp = gr.resize_window(frame_disp)
            cv2.imshow('Збір даних', frame_disp)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        self.collect_running = False
        self.collect_ready = False
        self.frames[CollectFrame].status_label.config(text=f"Збір завершено для '{gesture_name}'")
        self.frames[CollectFrame].btn_start.config(state=tk.NORMAL)
        self.frames[CollectFrame].btn_stop.config(state=tk.DISABLED)

    def stop_collect(self):
        self.collect_running = False
        self.collect_ready = False

    def select_gesture(self):
        idx = GestureListFrame.select_gesture_dialog(self.gestures, self.root, self.samples_per_gesture)
        if idx is not None:
            self.collect_gesture_idx = idx
            self.frames[CollectFrame].status_label.config(
                text=f"Готові до збору: '{self.gestures[self.collect_gesture_idx]}'"
            )
            messagebox.showinfo("Вибір жесту", f"Вибрано жест: {self.gestures[self.collect_gesture_idx]}")

    # --------- Gesture Recognition --------
    def start_recognize(self):
        if self.recognize_running:
            return
        if os.path.exists("gesture_rnn_model.h5"):
            self.model = load_model("gesture_rnn_model.h5")
            self.labels = self.gestures
        else:
            messagebox.showerror("Помилка", "Модель не знайдена. Потрібно спершу натренувати модель.")
            return
        self.recognize_running = True
        self.recognize_thread = Thread(target=self.recognize_data_thread, daemon=True)
        self.recognize_thread.start()

    def recognize_data_thread(self):
        gr = self.gr_class(
            mode="recognize",
            gestures=self.gestures,
            font_path=FONT_PATH,
            window_scale=self.window_scale
        )

        cap = cv2.VideoCapture(0)
        sequence_buffer = deque(maxlen=self.sequence_length)

        gesture_name = ""
        top_text = ""       # тоже инициализируем
        expected_len = 126  # 2 руки × 21 точка × 3 координати

        while self.recognize_running:
            ret, frame = cap.read()
            if not ret:
                continue
            frame = cv2.flip(frame, 1)
            frame_orig = frame.copy()
            frame, hands_all = gr.process_frame(frame)

            vec = []
            if hands_all:
                for hand_landmarks in hands_all:
                    vec.extend(gr.extract_landmarks(hand_landmarks))
                if len(hands_all) == 1:
                    vec.extend([0.] * 63)
            else:
                # Коли рук немає, використовуйте нульовий вектор бажаної довжини
                vec = [0.] * expected_len

            if len(vec) != expected_len:
                print(f"Пропускаємо кадр через некоректну довжину вектора: {len(vec)} (очікувалося {expected_len})")
                continue  # Пропускаємо кадр

            sequence_buffer.append(vec)

            if len(sequence_buffer) == self.sequence_length:
                data = np.array(sequence_buffer).reshape(1, self.sequence_length, -1)
                try:
                    prediction_prob = self.model.predict(data, verbose=0)[0]
                    pred_idx = np.argmax(prediction_prob)
                    confidence = prediction_prob[pred_idx]
                    if confidence < self.confidence_threshold:
                        gesture_name = ""
                    else:
                        gesture_name = self.labels[pred_idx]

                    top_indices = np.argsort(prediction_prob)[::-1]
                    top_text_lines = []
                    for i in top_indices[:3]:
                        label = self.labels[i]
                        conf_percent = prediction_prob[i] * 100
                        top_text_lines.append(f"{label} {conf_percent:.0f}%")
                    top_text = "\n".join(top_text_lines)
                except Exception:
                    gesture_name = ""
                    top_text = ""

            frame_disp = gr.show_on_screen(frame_orig, text=None, subtext=None)
            frame_disp = gr.resize_window(frame_disp)

            frame_pil = Image.fromarray(cv2.cvtColor(frame_disp, cv2.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(frame_pil)
            font_main = ImageFont.truetype(FONT_PATH, 36)
            font_small = ImageFont.truetype(FONT_PATH, 22)
            draw.text((10, 10), f"Розпізнано: {gesture_name}", font=font_main, fill=(0, 0, 0))

            lines = top_text.split('\n')
            top_text_line = "   ".join(lines)
            main_w, _ = font_main.getbbox(f"Розпізнано: {gesture_name}")[2:4]
            draw.text((10 + main_w + 30, 20), top_text_line, font=font_small, fill=(0, 0, 0))

            frame_disp = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)

            cv2.imshow('Розпізнавання жестів', frame_disp)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        self.recognize_running = False
        self.show_frame(MainMenu)

    def stop_recognize(self):
        self.recognize_running = False

    def show_gesture_list(self):
        # self.show_gestures_popup(self.gestures, self.root, self.samples_per_gesture)
        self.frames[GestureListFrame].refresh_list()  # щоб оновити список
        self.show_frame(GestureListFrame)
        
    @staticmethod
    def show_gestures_popup(gestures, parent, samples_required=100):
        dialog = tk.Toplevel(parent)
        dialog.title("Список жестів")
        # dialog.geometry("800x600")

        frame = tk.Frame(dialog)
        frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        scrollbar = tk.Scrollbar(frame, orient=tk.VERTICAL)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        listbox = tk.Listbox(frame, font=("Arial", 12), yscrollcommand=scrollbar.set)
        scrollbar.config(command=listbox.yview)
        listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        for idx, gesture in enumerate(gestures, start=1):
            gesture_dir = os.path.join('dataset', gesture)
            collected = len(glob.glob(os.path.join(gesture_dir, '*.npy')))
            listbox.insert(tk.END, f"{idx}. {gesture}: [{collected}/{samples_required}]")

        ttk.Button(dialog, text="Закрити", command=dialog.destroy).pack(pady=10)

        dialog.transient(parent)
        dialog.grab_set()
        parent.wait_window(dialog)

    # ----------- Sentence builder mode -----------
    def start_sentence_mode(self):
        self.sentence_running = False
        self.frames[SentenceFrame].update_sentence("")
        self.builded_sentence = []
        self.sentence_last_word = None

    def start_sentence_capture(self):
        if self.sentence_running:
            return
        if os.path.exists("gesture_rnn_model.h5"):
            self.model = load_model("gesture_rnn_model.h5")
            self.labels = self.gestures
        else:
            messagebox.showerror("Помилка", "Модель не знайдена. Потрібно спершу натренувати модель.")
            self.frames[SentenceFrame].btn_start.config(state=tk.NORMAL)
            return
        self.sentence_running = True
        self.sentence_thread = Thread(target=self.sentence_mode_thread, daemon=True)
        self.sentence_thread.start()

    def stop_sentence_mode(self):
        self.sentence_running = False

    def remove_last_word_sentence(self):
        if self.builded_sentence:
            self.builded_sentence.pop()
            self.frames[SentenceFrame].update_sentence(" ".join(self.builded_sentence))

    def clear_sentence(self):
        self.builded_sentence = []
        self.frames[SentenceFrame].update_sentence("")

    def sentence_mode_thread(self):
        gr = self.gr_class(
            mode="recognize",
            gestures=self.gestures,
            font_path=FONT_PATH,
            window_scale=self.window_scale
        )
        cap = cv2.VideoCapture(0)
        sequence_buffer = deque(maxlen=self.sequence_length)
        last_gesture = ""
        gesture_hold_count = 0
        required_hold_frames = 10
        last_add_time = time.time()
        min_interval = 1.5

        while self.sentence_running:
            ret, frame = cap.read()
            if not ret:
                continue
            frame = cv2.flip(frame, 1)
            frame_orig = frame.copy()
            frame, hands_all = gr.process_frame(frame)

            vec = []
            if hands_all:
                for hand_landmarks in hands_all:
                    vec.extend(gr.extract_landmarks(hand_landmarks))
                if len(hands_all) == 1:
                    vec.extend([0.] * 63)
            else:
                vec = [0.] * 126

            expected_len = 126  # длина вектора признаков (2 руки × 21 точка × 3 координаты)

            if len(vec) != expected_len:
                print(f"Пропускаємо кадр через некоректну довжину вектора: {len(vec)} (очікувалось {expected_len})")
                continue  # пропускаем этот кадр — не добавляем
            sequence_buffer.append(vec)         
            gesture_name = ""
            top_text_line = ""

            if len(sequence_buffer) == self.sequence_length:
                data = np.array(sequence_buffer).reshape(1, self.sequence_length, -1)
                try:
                    prediction_prob = self.model.predict(data, verbose=0)[0]
                    pred_idx = np.argmax(prediction_prob)
                    confidence = prediction_prob[pred_idx]
                    if confidence < self.confidence_threshold:
                        gesture_name = ""
                        top_text_line = ""
                    else:
                        gesture_name = self.labels[pred_idx]

                        top_indices = np.argsort(prediction_prob)[::-1]
                        parts = []
                        for i in top_indices[:3]:
                            label = self.labels[i]
                            conf_percent = prediction_prob[i] * 100
                            parts.append(f"{label} {conf_percent:.0f}%")
                        top_text_line = "   ".join(parts)
                except Exception:
                    gesture_name = ""
                    top_text_line = ""

            if gesture_name == last_gesture and gesture_name != "":
                gesture_hold_count += 1
            else:
                gesture_hold_count = 1
                last_gesture = gesture_name

            now = time.time()
            if (gesture_name and
                (len(self.builded_sentence) == 0 or gesture_name != self.builded_sentence[-1]) and
                gesture_hold_count >= required_hold_frames and
                (now - last_add_time) > min_interval):
                self.builded_sentence.append(gesture_name)
                self.frames[SentenceFrame].update_sentence(" ".join(self.builded_sentence))
                last_add_time = now
                gesture_hold_count = 0

            frame_disp = gr.show_on_screen(
                frame_orig,
                text=f"Розпізнано: {gesture_name}",
                subtext=top_text_line,
                font_size=36
            )
            frame_disp = gr.resize_window(frame_disp)
            cv2.imshow('Побудова речення', frame_disp)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        self.sentence_running = False
        self.frames[SentenceFrame].btn_start.config(state=tk.NORMAL)
        
# Графіки навчання
    def plot_training_history(self, history):
    
        plt.figure(figsize=(12, 5))
        # Потери
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Втрата (train)')
        if 'val_loss' in history.history:
            plt.plot(history.history['val_loss'], label='Втрата (val)')
        plt.title('Функція втрат')
        plt.xlabel('Епоха')
        plt.ylabel('Втрати')
        plt.legend()
        plt.grid()

        # Точність
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Точність (train)')
        if 'val_accuracy' in history.history:
            plt.plot(history.history['val_accuracy'], label='Точність (val)')
        plt.title('Точність')
        plt.xlabel('Епоха')
        plt.ylabel('Точність')
        plt.legend()
        plt.grid()

        plt.tight_layout()
        plt.show()

In [6]:
# ==== Frames ====

class MainMenu(tk.Frame):
    def __init__(self, parent, app):
        super().__init__(parent)
        self.app = app
        tk.Label(self, text="Система розпізнавання жестів", font=("Arial", 18, "bold")).pack(pady=15)
        ttk.Button(self, text="Збір даних", command=lambda: app.show_frame(CollectFrame)).pack(pady=7)
        ttk.Button(self, text="Тренувати модель", command=app.train_model).pack(pady=7)
        ttk.Button(self, text="Заново натренувати модель", command=app.retrain_model_from_scratch).pack(pady=7)
        ttk.Button(self, text="Почати розпізнавання", command=lambda: [app.show_frame(RecognizeFrame), app.start_recognize()]).pack(pady=7)
        ttk.Button(self, text="Побудова речення", command=lambda: [app.show_frame(SentenceFrame), app.start_sentence_mode()]).pack(pady=7)
        ttk.Button(self, text="Показати жести", command=app.show_gesture_list).pack(pady=7)
        ttk.Button(self, text="Вихід", command=app.root.destroy).pack(pady=10)

class CollectFrame(tk.Frame):
    def __init__(self, parent, app):
        super().__init__(parent)
        self.app = app
        tk.Label(self, text="Збір даних", font=("Arial", 15, "bold")).pack(pady=10)
        self.status_label = tk.Label(self, text="Готові до збору", font=("Arial", 12))
        self.status_label.pack(pady=5)
        delay_frame = tk.Frame(self)
        delay_frame.pack(pady=10)
        tk.Label(delay_frame, text="Затримка між зразками (с):").pack(side=tk.LEFT)
        self.delay_var = tk.DoubleVar(value=self.app.delay_between_samples)
        self.delay_entry = ttk.Entry(delay_frame, width=6, textvariable=self.delay_var)
        self.delay_entry.pack(side=tk.LEFT, padx=5)
        apply_btn = ttk.Button(delay_frame, text="Застосувати", command=self.apply_delay)
        apply_btn.pack(side=tk.LEFT)
        button_frame = tk.Frame(self)
        button_frame.pack(pady=10)
        self.btn_start = ttk.Button(button_frame, text="Пуск", command=self.start_collect)
        self.btn_start.pack(side=tk.LEFT, padx=8)
        self.btn_stop = ttk.Button(button_frame, text="Стоп", command=self.stop_collect, state=tk.DISABLED)
        self.btn_stop.pack(side=tk.LEFT, padx=8)
        ttk.Button(button_frame, text="Обрати інший жест", command=app.select_gesture).pack(side=tk.LEFT, padx=8)
        ttk.Button(button_frame, text="Головне меню", command=lambda: [app.stop_collect(), app.show_frame(MainMenu)]).pack(side=tk.LEFT, padx=8)

    def apply_delay(self):
        try:
            val = float(self.delay_var.get())
            if val < 0:
                raise ValueError("Затримка повинна бути не від'ємною")
            self.app.delay_between_samples = val
            messagebox.showinfo("Інфо", f"Затримка між зразками встановлена на {val} секунд")
        except Exception as e:
            messagebox.showerror("Помилка", f"Невірне значення: {e}")

    def start_collect(self):
        self.status_label.config(text=f"Йде збір... '{self.app.gestures[self.app.collect_gesture_idx]}'")
        self.btn_start.config(state=tk.DISABLED)
        self.btn_stop.config(state=tk.NORMAL)
        self.app.collect_ready = True
        self.app.start_collect(self.app.collect_gesture_idx)

    def stop_collect(self):
        self.status_label.config(text="Збір зупинено")
        self.btn_start.config(state=tk.NORMAL)
        self.btn_stop.config(state=tk.DISABLED)
        self.app.stop_collect()


class RecognizeFrame(tk.Frame):
    def __init__(self, parent, app):
        super().__init__(parent)
        self.app = app
        tk.Label(self, text="Розпізнавання жестів", font=("Arial", 15, "bold")).pack(pady=10)
        button_frame = tk.Frame(self)
        button_frame.pack(pady=10)
        ttk.Button(button_frame, text="Список жестів", command=self.app.show_gesture_list).pack(side=tk.LEFT, padx=8)
        ttk.Button(button_frame, text="Головне меню", command=lambda: [self.app.stop_recognize(), self.app.show_frame(MainMenu)]).pack(side=tk.LEFT, padx=8)

class GestureListFrame(tk.Frame):
    def __init__(self, parent, app):
        super().__init__(parent)
        self.app = app
        # self.app.root.minsize(650, 550)

        tk.Label(self, text="Список жестів (прогрес збору)", font=("Arial", 15, "bold")).pack(pady=10)

        # Поле пошуку
        search_frame = tk.Frame(self)
        search_frame.pack(pady=5, padx=10, fill='x')

        tk.Label(search_frame, text="Пошук жесту:").pack(side=tk.LEFT)
        self.search_var = tk.StringVar()
        self.search_var.trace_add("write", self.on_search)
        self.search_entry = ttk.Entry(search_frame, textvariable=self.search_var)
        self.search_entry.pack(side=tk.LEFT, fill='x', expand=True, padx=5)

        # Список із скролбаром
        frame_list = tk.Frame(self)
        frame_list.pack(fill=tk.BOTH, expand=True)

        self.scrollbar = tk.Scrollbar(frame_list)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.listbox = tk.Listbox(frame_list, width=80, height=25, font=("Arial", 12), yscrollcommand=self.scrollbar.set)
        self.listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.scrollbar.config(command=self.listbox.yview)

        # Повний список для фільтрування
        self.full_list = []
        self.refresh_list()

        # Кнопки
        btn_frame = tk.Frame(self)
        btn_frame.pack(pady=10)
        ttk.Button(btn_frame, text="Оновити", command=self.refresh_list).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="Головне меню", command=lambda: app.show_frame(MainMenu)).pack(side=tk.LEFT, padx=5)

    def refresh_list(self):
        self.listbox.delete(0, tk.END)
        self.full_list.clear()
        for idx, gesture in enumerate(self.app.gestures, 1):
            gesture_dir = os.path.join('dataset', gesture)
            collected = len(glob.glob(os.path.join(gesture_dir, '*.npy')))
            required = self.app.samples_per_gesture
            line = f"{idx}. {gesture} [{collected}/{required}]"
            self.full_list.append(line)
        self.update_listbox(self.full_list)

    def update_listbox(self, items):
        self.listbox.delete(0, tk.END)
        for item in items:
            self.listbox.insert(tk.END, item)
        if not items:
            messagebox.showinfo("Пошук", "Жест не знайдено.")

    def on_search(self, *args):
        query = self.search_var.get().lower().strip()
        if not query:
            self.update_listbox(self.full_list)
            return

        filtered = [item for item in self.full_list if query in item.lower()]
        self.update_listbox(filtered)

    @staticmethod
    def select_gesture_dialog(gestures, parent, samples_required):
        dialog = tk.Toplevel(parent)
        dialog.title("Вибір жесту")
        dialog.geometry("500x600")  # Велике вікно, аналогічне "Показати жести"
    
        # Поле пошуку
        search_frame = tk.Frame(dialog)
        search_frame.pack(pady=5, padx=10, fill='x')

        tk.Label(search_frame, text="Пошук жесту:").pack(side=tk.LEFT)
        search_var = tk.StringVar()
        search_entry = ttk.Entry(search_frame, textvariable=search_var)
        search_entry.pack(side=tk.LEFT, fill='x', expand=True, padx=5)

        # Список з прокруткою
        frame_list = tk.Frame(dialog)
        frame_list.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        scrollbar = tk.Scrollbar(frame_list, orient=tk.VERTICAL)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        listbox = tk.Listbox(frame_list, font=("Arial", 12), yscrollcommand=scrollbar.set)
        scrollbar.config(command=listbox.yview)
        listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Функція оновлення вмісту списку з урахуванням пошуку
        full_list = []
        for i, gesture in enumerate(gestures, 1):
            gesture_dir = os.path.join('dataset', gesture)
            collected = len(glob.glob(os.path.join(gesture_dir, '*.npy')))
            line = f"{i}. {gesture} [{collected}/{samples_required}]"
            full_list.append(line)

        def update_listbox(items):
            listbox.delete(0, tk.END)
            for item in items:
                listbox.insert(tk.END, item)
            if not items:
                messagebox.showinfo("Пошук", "Жест не знайдено.")

        update_listbox(full_list)

        def on_search(*args):
            query = search_var.get().lower().strip()
            if not query:
                update_listbox(full_list)
                return
            filtered = [item for item in full_list if query in item.lower()]
            update_listbox(filtered)

        search_var.trace_add('write', on_search)

        selected_index = {'idx': None}

        def on_select():
            selection = listbox.curselection()
            if selection:
                selected_index['idx'] = selection[0]
                dialog.destroy()

        btn_frame = tk.Frame(dialog)
        btn_frame.pack(pady=5)

        ttk.Button(btn_frame, text="Вибрати", command=on_select).pack(side=tk.LEFT, padx=10)
        ttk.Button(btn_frame, text="Відмінити", command=dialog.destroy).pack(side=tk.RIGHT, padx=10)

        listbox.bind('<Double-Button-1>', lambda e: on_select())

        dialog.transient(parent)
        dialog.grab_set()
        parent.wait_window(dialog)

        return selected_index['idx']

    @staticmethod
    def show_gestures_popup(gestures, parent, samples_required=100):
        dialog = tk.Toplevel(parent)
        dialog.title("Список жестів")
        # dialog.geometry("600x600")   # Збільшений розмір вікна

        frame = tk.Frame(dialog)
        frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        scrollbar = tk.Scrollbar(frame, orient=tk.VERTICAL)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        listbox = tk.Listbox(frame, font=("Arial", 12), yscrollcommand=scrollbar.set)
        scrollbar.config(command=listbox.yview)
        listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        for i, gesture in enumerate(gestures, 1):
            gesture_dir = os.path.join('dataset', gesture)
            collected = len(glob.glob(os.path.join(gesture_dir, '*.npy')))
            listbox.insert(tk.END, f"{i}. {gesture}: [{collected}/{samples_required}]")

        ttk.Button(dialog, text="Закрити", command=dialog.destroy).pack(pady=10)

        dialog.transient(parent)
        dialog.grab_set()
        parent.wait_window(dialog)

# ----------------- Новый Frame для "Побудова речення" ---------------
class SentenceFrame(tk.Frame):
    def __init__(self, parent, app):
        super().__init__(parent)
        self.app = app
        tk.Label(self, text="Побудова речення (жестами)", font=("Arial", 15, "bold")).pack(pady=10)

        self.sentence_var = tk.StringVar()
        self.sentence_label = tk.Label(self, textvariable=self.sentence_var, font=("Arial", 16), bg="#f5f5dc", wraplength=480, height=3, anchor="w", justify="left")
        self.sentence_label.pack(pady=12, padx=14, fill="x")

        ctrl_frame = tk.Frame(self)
        ctrl_frame.pack(pady=4)
        ttk.Button(ctrl_frame, text="Прибрати останній жест", command=self.remove_last_word).pack(side=tk.LEFT, padx=8)
        ttk.Button(ctrl_frame, text="Прибрати речення", command=self.clear_sentence).pack(side=tk.LEFT, padx=8)
        ttk.Button(ctrl_frame, text="Головне меню", command=self.exit_to_menu).pack(side=tk.LEFT, padx=8)

        self.btn_start = ttk.Button(self, text="Старт", command=self.start_sentence_capture)
        self.btn_start.pack(pady=12)

    def update_sentence(self, sentence):
        self.sentence_var.set(sentence)

    def start_sentence_capture(self):
        self.btn_start.config(state=tk.DISABLED)
        self.app.start_sentence_capture()

    def remove_last_word(self):
        self.app.remove_last_word_sentence()

    def clear_sentence(self):
        self.app.clear_sentence()

    def exit_to_menu(self):
        self.app.stop_sentence_mode()
        self.app.show_frame(MainMenu)
        self.btn_start.config(state=tk.NORMAL)

class PrintTwoDecimalsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        loss = logs.get('loss', 0)
        acc = logs.get('accuracy', 0)
        val_loss = logs.get('val_loss')
        val_acc = logs.get('val_accuracy')
        msg = f"Epoch {epoch+1}: loss: {loss:.2f}, accuracy: {acc:.2f}"
        if val_loss is not None and val_acc is not None:
            msg += f", val_loss: {val_loss:.2f}, val_accuracy: {val_acc:.2f}"
        print(msg)

In [7]:
if __name__ == "__main__":
    app = AppGUI(
        GestureRecognition,
        GESTURES,
        SAMPLES_PER_GESTURE,
        DELAY_BETWEEN_SAMPLES,
        CONFIDENCE_THRESHOLD,
        SEQUENCE_LENGTH,
        window_scale=1.5
    )
    app.run()